In [276]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import argparse
import igraph
from igraph import Graph, EdgeSeq
np.random.seed(0)

# States
$x_{min} = -x_{max}\\
v_{min} = -v_{max}$

State: 
$s = \begin{bmatrix} x\\ 
                v \end{bmatrix}$
            
The state space is continuous.



In [277]:
# States
x = np.zeros(2)
x_min = -5
x_max = -x_min
v_min = -1
v_max = -v_min

def check_bounds(s):
    return np.abs(s[0,0]) < x_max and np.abs(s[1,0]) < v_max

# Actions
Continuous action $a \in [-1, 1]$ corresponding to acceleration.

In [278]:
# Actions
a = 0 # initialize the action to zero for now
a_min = -1
a_max = 1

# System Dynamics
Assuming continuous time,

$\dot{s} = A * s + B * u$

$y = C*s + D*u$

$A = \begin{bmatrix}
      			0 & 1 \\
                0 & 0 
			\end{bmatrix}$

$B = \begin{bmatrix}
      			0\\
                1 
			\end{bmatrix}$

$u = a$

$C = \begin{bmatrix}
      			1 & 0 \\
                0 & 1 
			\end{bmatrix}$

$D = \begin{bmatrix}
      			0 \\
                0 
			\end{bmatrix}$


Assuming discrete time,

$ x(t+dt) = a(t)*dt^2/2 + v(t) * dt + x(t) $

$v(t+dt) = a(t)* dt +v(t)$

$s(t+dt) = A * s(t) + B * u(t)$

with dt = 1,

$\begin{bmatrix}
      			x(t+1)\\
                v(t+1)
			\end{bmatrix} 
            = \begin{bmatrix}
      			1 & 1 \\
                0 & 1 
			\end{bmatrix} \begin{bmatrix}
				x(t)\\
                v(t)
			\end{bmatrix}
            + \begin{bmatrix}
      			1/2\\
                1 
			\end{bmatrix}u$


$A = \begin{bmatrix}
      			1 & 1 \\
                0 & 1 
			\end{bmatrix}$

$B = \begin{bmatrix}
      			1/2\\
                1 
			\end{bmatrix}$

$u = a$

$C = \begin{bmatrix}
      			1 & 0 \\
                0 & 1 
			\end{bmatrix}$

$D = \begin{bmatrix}
      			0 \\
                0 
			\end{bmatrix}$

In [279]:
# System Dynamics
A = np.array([
    [1, 1],
    [0, 1]
    ])
B = np.array([
    [1/2],
    [1]
    ])
C = np.eye(2)
D = np.zeros((2,1))

def step(s, u):
    return np.dot(A,s) + np.dot(B,u)

# Goal State
$s_f = \begin{bmatrix}
				x\\
                v
			\end{bmatrix} = \begin{bmatrix}
				0\\
                0
			\end{bmatrix}$
            
We expand the goal state to a goal space $\mathcal{S}_f$ by allowing some deviation from $x=0$ and $v=0$.


In [280]:
s_f = np.array([[0],[0]])
print("s_f:\n", s_f)
def goal_reached(s, s_f, dx=0.1, dv=0.1):
#     return np.linalg.norm(s - s_f) <= np.linalg.norm(np.array([dx, dv]))
    return (np.abs(s[0,0] - s_f[0,0]) < dx) and  (np.abs(s[1,0] - s_f[1,0]) < dv)

# Randomized Initial State
s_i = np.zeros((2,1)) 
s_i[0,0] = np.random.uniform(x_min, x_max)
s_i[1,0] = np.random.uniform(v_min, v_max)
print("s_i:\n", s_i)

s_f:
 [[0]
 [0]]
s_i:
 [[0.48813504]
 [0.43037873]]


# Random Sample
Every iteration we need to randomly sample some $s_{rand}$ from our configuration space to drive towards and expand our tree. With some probability, we will sample our goal state, $s_f$, to explicitly extend the tree in the direction of the goal.
Define the random_sample() function

In [281]:
def random_sample(p=0.05):
    theta = np.random.sample()
    if theta < p:
        return s_f
    s_rand = np.zeros((2,1))
    s_rand[0,0] = np.random.uniform(x_min, x_max)
    s_rand[1,0] = np.random.uniform(v_min, v_max)
    return s_rand

# Defining the Graph
We need to initialize the list of vertices, $V$, with our $s_i$ and the list of edges, $E$, to an empty list.
The nearest_vertex function finds the nearest vertex in $V$ to some vertex $v$.

In [282]:
V = [s_i] # initialize the list of vertices
E = [] # initialize the list of edges
def nearest_vertex(v, V):
    return min([(x,np.linalg.norm(x-v),i) for i, x in enumerate(V)], key = lambda x: x[1])

# Drive To Function

In [283]:
def drive_to(s_now, s_r):
    # drive from s_now to s_r and return the resulting state
    u = -0.2 if s_now[0,0] < s_r[0,0] else 0.2
    return step(s_now, u)
    

# Run the Simulation

In [284]:
def demo():
    E = []
    V = [s_i]
    print("E: ", E)
    print("V: ", V)
    
    while True:
        s_rand = random_sample()
#         print("s_rand: ", s_rand)
        nearest_vert, dist, i = nearest_vertex(s_rand, V)
#         print("nearest vertex: ", nearest_vert)
        s_new = drive_to(nearest_vert, s_rand)
#         print("s_new: ", s_new)
        if check_bounds(s_new):
            V.append(s_new)
            E.append((nearest_vert, s_new))
#         print("new edge: ", E[-1])
            if goal_reached(s_new, s_f):
                return V,E

In [285]:
V, E = demo()


E:  []
V:  [array([[0.48813504],
       [0.43037873]])]


# Plotting
Some plotly examples:
https://chart-studio.plotly.com/~empet/14676/bcl-2-gene-family-tree/#plot

https://chart-studio.plotly.com/~empet/14307/a-radial-tree/#/

https://chart-studio.plotly.com/~empet/14305/a-tree-with-improved-walker-layout-1/#/

https://plotly.com/python/tree-plots/

https://plotly.com/python/animations/

In [296]:
# Xe = [x, y for x,y in E]
Xe = []
Ye = []
for edge in E:
#     print(E[j])
#     print(edge)
    Xe+=[edge[0][0,0],edge[1][0,0], None]
    Ye+=[edge[0][1,0],edge[1][1,0], None]

Xn = [x[0,0] for x in V]
Yn = [x[1,0] for x in V]
Xi = [s_i[0,0]]
Yi = [s_i[1,0]]
Xf = [s_f[0,0]]
Yf = [s_f[1,0]]

fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                         name='Edges',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='Vertices',
                  marker=dict(symbol='circle-dot',
                                size=2,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))
fig.add_trace(go.Scatter(x=Xi,
                  y=Yi,
                  mode='markers',
                  name='Start',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#f67c07',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))

fig.add_trace(go.Scatter(x=Xf,
                  y=Yf,
                  mode='markers',
                  name='Goal',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#54ca41',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.5
                  ))
fig.update_layout(title={'text':"RRT",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="Position", yaxis_title="Velocity")
fig.show()


# Potential Field
$\phi(x)$ s.t. $v(t+1) = v(t) + a - \frac{d}{dx}\phi(x)$



# Implementation
Github repo here: https://github.com/eczy/rrt-2dof-1dm